# Recommandation System
# DIA CORE 4
## MAHMOUDI Amir
## LAHBABI Yassine

#### Librairies Import & CSV

In [2]:
import pandas as pd
import re
import math
import numpy as np
from numpy import linalg as LA

In [3]:
anime = pd.read_csv('myAnimeListDataset.csv')

In [4]:
anime

,animeID,name,premiered,genre,type,episodes,producer,licensor,studio,source,scored,scoredBy,members
0,1,Cowboy Bebop,Spring 1998,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",TV,26,['Bandai Visual'],"['Funimation', 'Bandai Entertainment']",['Sunrise'],Original,8.81,363889,704490
1,5,Cowboy Bebop: Tengoku no Tobira,None,"['Action', 'Space', 'Drama', 'Mystery', 'Sci-Fi']",Movie,1,"['Sunrise', 'Bandai Visual']",['Sony Pictures Entertainment'],['Bones'],Original,8.41,111187,179899
2,6,Trigun,Spring 1998,"['Action', 'Sci-Fi', 'Adventure', 'Comedy', 'D...",TV,26,['Victor Entertainment'],"['Funimation', 'Geneon Entertainment USA']",['Madhouse'],Manga,8.31,197451,372709
3,7,Witch Hunter Robin,Summer 2002,"['Action', 'Magic', 'Police', 'Supernatural', ...",TV,26,['Bandai Visual'],"['Funimation', 'Bandai Entertainment']",['Sunrise'],Original,7.34,31875,74889
4,8,Beet the Vandel Buster,Fall 2004,"['Adventure', 'Fantasy', 'Shounen', 'Supernatu...",TV,52,[],[],['Toei Animation'],Manga,7.04,4757,11247
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13363,37829,Centaur no Noyami season 2,None,"['Comedy', 'Fantasy', 'Seinen', 'Slice of Life']",TV,12,[],[],[],Manga,0.00,0,0
13364,37832,Tenkū no Shiro Rapyuta,None,"['Adventure', 'Fantasy', 'Romance', 'Sci-Fi']",Movie,1,[],[],[],Unknown,0.00,0,0
13365,37833,Qíjī xīng,None,['Shounen'],Special,12,[],[],[],Original,0.00,0,0
13366,37834,Godzilla Kingdom,None,"['Comedy', 'Kids', 'Music']",TV,224,[],[],[],Unknown,0.00,0,0


#### Extract information from dataframe

In [6]:
def extractor(line):
    linedata = anime.iloc[line]
    # Title and date extraction
    chain = linedata[1].split(" ")

    # Title
    title = ""
    for i in range (len(chain)-1):
        title += chain[i]
        if i != len(chain)-2:
            title += " "
    genrelist = linedata[3]
    type_anime=linedata[4]
    return str(linedata[0]), title, type_anime, eval(genrelist)

In [7]:
extractor(0)

('1',
 'Cowboy',
 'TV',
 ['Action', 'Adventure', 'Comedy', 'Drama', 'Sci-Fi', 'Space'])

#### Transform genres in columns

In [8]:
animeId = []
title = []
type_animes = []
genres=[]

for i in range(anime.shape[0]):
    mid, tit, typ, gen = extractor(i)
    animeId.append(mid)
    title.append(tit)
    type_animes.append(typ)
    genres.append(gen)

In [9]:
new_columns= list(anime.columns)
anime_norm=pd.DataFrame(columns=new_columns)
anime_norm['animeID'] = animeId
anime_norm['name'] = title
anime_norm['type'] = type_animes
anime_norm = anime_norm.drop(columns=['genre'])

genres_df = pd.DataFrame({"genre":list(genres)})
anime_norm = anime_norm.join(genres_df)
genres_ohe = pd.get_dummies(anime_norm.genre.apply(pd.Series).stack()).sum(level=0)
anime_norm = anime_norm.join(genres_ohe)


In [10]:
anime_norm

,animeID,name,premiered,type,episodes,producer,licensor,studio,source,scored,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
0,1,Cowboy,NaN,TV,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,Cowboy Bebop: Tengoku no,NaN,Movie,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,,NaN,TV,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7,Witch Hunter,NaN,TV,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,8,Beet the Vandel,NaN,TV,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13363,37829,Centaur no Noyami season,NaN,TV,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13364,37832,Tenkū no Shiro,NaN,Movie,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13365,37833,Qíjī,NaN,Special,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13366,37834,Godzilla,NaN,TV,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Normalization

In [11]:
def normalize(df):
    for index , row in df.iterrows(): 
        nbofgenres = len(df.iloc[index,12])
        for i in range(len(df.iloc[index,13:])):
               if df.iloc[index,13+i]==1:
                    df.iloc[index,13+i] = df.iloc[index,13+i]/math.sqrt(nbofgenres)

In [12]:
normalize(anime_norm)
anime_norm=anime_norm.drop(columns=["premiered","type","episodes","producer","licensor","studio","source","scored","scoredBy","members"])
anime_norm

,animeID,name,genre,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
0,1,Cowboy,"[Action, Adventure, Comedy, Drama, Sci-Fi, Space]",0.408248,0.408248,0.0,0.408248,0.0,0.0,0.408248,...,0.0,0.0,0.408248,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,5,Cowboy Bebop: Tengoku no,"[Action, Space, Drama, Mystery, Sci-Fi]",0.447214,0.000000,0.0,0.000000,0.0,0.0,0.447214,...,0.0,0.0,0.447214,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,6,,"[Action, Sci-Fi, Adventure, Comedy, Drama, Sho...",0.408248,0.408248,0.0,0.408248,0.0,0.0,0.408248,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,7,Witch Hunter,"[Action, Magic, Police, Supernatural, Drama, M...",0.408248,0.000000,0.0,0.000000,0.0,0.0,0.408248,...,0.0,0.0,0.000000,0.0,0.0,0.408248,0.0,0.0,0.0,0.0
4,8,Beet the Vandel,"[Adventure, Fantasy, Shounen, Supernatural]",0.000000,0.500000,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.500000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13363,37829,Centaur no Noyami season,"[Comedy, Fantasy, Seinen, Slice of Life]",0.000000,0.000000,0.0,0.500000,0.0,0.0,0.000000,...,0.0,0.5,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
13364,37832,Tenkū no Shiro,"[Adventure, Fantasy, Romance, Sci-Fi]",0.000000,0.500000,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
13365,37833,Qíjī,[Shounen],0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
13366,37834,Godzilla,"[Comedy, Kids, Music]",0.000000,0.000000,0.0,0.577350,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


#### User dataframe

Here we will create a datafram using the animes our user as seen and the ratings on each animes mentionned

In [47]:
new_columns= ["rating"]+list(anime_norm.columns)
user_input=pd.DataFrame(columns=new_columns)

In [48]:
print("Enter an anime name you've seen or enter Exit to quit")
name_input=input()
print("Enter the rate (scale 0 to 10) for "+str(name_input))
rating_input=input()
while(name_input!="Exit"):
    anime_input=anime_norm[anime_norm['name']==name_input]
    user_input=user_input.append(anime_input)
    user_input.loc[user_input.name == name_input, 'rating'] = rating_input
    print("Enter an anime name you've seen or enter Exit to quit")
    name_input=input()
    if(name_input!="Exit"):
        print("Enter the rate for "+str(name_input))
        rating_input=input()
user_input

Enter an anime name you've seen or enter Exit to quit
Witch Hunter
Enter the rate (scale 0 to 10) for Witch Hunter
10
Enter an anime name you've seen or enter Exit to quit
Cowboy
Enter the rate for Cowboy
9
Enter an anime name you've seen or enter Exit to quit
Exit


,rating,animeID,name,genre,Action,Adventure,Cars,Comedy,Dementia,Demons,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
3,10,7,Witch Hunter,"[Action, Magic, Police, Supernatural, Drama, M...",0.408248,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.408248,0.0,0.0,0.0,0.0
0,9,1,Cowboy,"[Action, Adventure, Comedy, Drama, Sci-Fi, Space]",0.408248,0.408248,0.0,0.408248,0.0,0.0,...,0.0,0.0,0.408248,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


The cell below we will sum the movies with the value of each genre to obtain a vector

In [50]:
genre=['Action', 'Adventure', 'Cars', 'Comedy', 'Dementia', 'Demons', 'Drama', 'Ecchi', 'Fantasy', 'Game', 'Harem', 'Hentai', 'Historical', 'Horror', 'Josei', 'Kids', 'Magic', 'Martial Arts', 'Mecha', 'Military', 'Music', 'Mystery', 'Parody', 'Police', 'Psychological', 'Romance', 'Samurai', 'School', 'Sci-Fi', 'Seinen', 'Shoujo', 'Shoujo Ai', 'Shounen', 'Shounen Ai', 'Slice of Life', 'Space', 'Sports', 'Super Power', 'Supernatural', 'Thriller', 'Vampire', 'Yaoi', 'Yuri']
sums = list()
print('yes')
for i in range(4,47):
    sum = 0
    for index, row in user_input.iterrows():
        sum += row[i] * float(row['rating'])
    sums.append(sum)
user_compute=pd.DataFrame([sums],columns=genre)
user_compute

yes


,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
0,7.756718,3.674235,0.0,3.674235,0.0,0.0,7.756718,0.0,0.0,0.0,...,0.0,0.0,3.674235,0.0,0.0,4.082483,0.0,0.0,0.0,0.0


In [54]:
best_recommanded_animes =[]
list_movie_recommandations=[]
for index,anime in anime_norm.iloc[:200, :].iterrows():
    c=user_compute.values.tolist()
    sum_value=0
    normc=LA.norm(c)
    m=anime[3:]
    normm=LA.norm(m)
    for i in range(len(c)):
        sum_value+=(c[0][i]*m[i])
    heuristic=sum_value/(normc*normm)
    list_movie_recommandations.append(heuristic)
indexs = sorted(range(len(list_movie_recommandations)),key=lambda i: list_movie_recommandations[i])[-3:]
for e in indexs:
    best_recommanded_animes.append(anime_norm.iloc[e, :].animeID)
best_recommanded_animes

['152', '164', '198']

In [58]:
print("\nWe recommand you to see : \n")
answer=''
for anime_reco in best_recommanded_animes:
    answer += str(anime_norm[anime_norm["animeID"]==anime_reco].name.to_string(index=False))
    answer += "\t|\t"
index+=1
print(answer)


We recommand you to see : 

 Solty	|	 Mononoke	|	 Speed	|	
